In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pathlib
import plotly
import plotly_express as px
import plotly.graph_objs as go
import iplot
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import umap
from sklearn import preprocessing

In [2]:
nELISA_data_input_path = pathlib.Path("../Data/clean/nELISA_Data_UCA1_2023.04.11.csv")
MetaData_input_path = pathlib.Path("../Data/clean/Metadata_UCA1_2023.04.11.csv")

nELISA_data_all = pd.read_csv(nELISA_data_input_path)
MetaData = pd.read_csv(MetaData_input_path)

In [3]:
# nELISA_data = pd.merge(MetaData, nELISA_data_all, left_on='plate_position',right_on='plate_position', how='inner')
nELISA_data = pd.merge(MetaData, nELISA_data_all, on="plate_position", how="right")
nELISA_data_values = nELISA_data.filter(like="NSU", axis=1)
nELISA_data_values = nELISA_data_values.astype("float")
print(nELISA_data_values.shape)
# pd.merge(MetaData, nELISA_data_all, on='plate_position',how='left').shape
nELISA_data_values.head()

(689, 187)


,Activin A [NSU],AITRL (GITR Ligand) [NSU],Amphiregulin [NSU],Amyloid beta [NSU],APRIL [NSU],BAFF [NSU],BCMA (TNFRSF17) [NSU],BDNF [NSU],BMP2 [NSU],BMP3 [NSU],...,TWEAK [NSU],uPA [NSU],VCAM-1 [NSU],VEGF Receptor 2 (Flk-1) [NSU],VEGF-A (165) [NSU],VEGF-C [NSU],VEGF-D [NSU],VEGFR-1 [NSU],WISP-1 (CCN4) [NSU],XCL1 (Lymphotactin) [NSU]
0,26.491764,-1.073383,25.817862,-2.435479,-2.556473,-1.259130,-0.043782,-0.661045,-1.508780,-2.046371,...,-0.728423,-0.681505,0.124148,2.617804,-2.210675,-1.433347,1.077095,5.713861,0.055628,-0.889479
1,24.943616,0.718127,26.598410,0.926190,-3.238686,-0.387113,3.650604,-0.227975,0.450686,0.104226,...,-1.296642,-1.387073,-0.140898,-1.198079,-0.128332,-1.550067,-2.169066,6.024132,0.340468,-0.844918
2,1.555501,0.268977,27.596171,-2.300495,-3.045141,-1.129626,2.458642,0.278998,0.062647,-1.156700,...,-0.746483,0.115185,-0.002896,-1.473596,0.235711,-1.449242,1.852248,-0.168032,0.961319,-0.363984
3,-0.929140,0.076314,26.313207,0.253148,-6.751763,-0.539164,-1.718383,1.443255,-0.341732,-0.719875,...,-2.232221,-1.126181,1.062176,1.542644,0.269262,-2.311105,2.088687,-0.306334,0.562805,-0.155359
4,0.398325,-0.130139,8.797693,-0.445542,0.371358,-0.462473,1.422635,-1.915471,-0.748352,-1.583011,...,-1.065390,-0.314291,0.171604,-0.137741,0.373863,-1.042447,-1.016304,-0.289143,0.373108,-0.804121


In [4]:
max_values = nELISA_data_values.max()
nELISA_data_values_sensor_max_norm = nELISA_data_values.div(max_values)
nELISA_data_values_sensor_max_norm.head()

,Activin A [NSU],AITRL (GITR Ligand) [NSU],Amphiregulin [NSU],Amyloid beta [NSU],APRIL [NSU],BAFF [NSU],BCMA (TNFRSF17) [NSU],BDNF [NSU],BMP2 [NSU],BMP3 [NSU],...,TWEAK [NSU],uPA [NSU],VCAM-1 [NSU],VEGF Receptor 2 (Flk-1) [NSU],VEGF-A (165) [NSU],VEGF-C [NSU],VEGF-D [NSU],VEGFR-1 [NSU],WISP-1 (CCN4) [NSU],XCL1 (Lymphotactin) [NSU]
0,0.780796,-0.402891,0.768750,-0.643275,-0.238687,-0.492554,-0.009443,-0.106441,-0.570024,-0.270803,...,-0.335835,-0.030973,0.041286,1.000000,-0.043905,-0.222372,0.237690,0.707199,0.016986,-0.218599
1,0.735167,0.269547,0.791992,0.244632,-0.302382,-0.151433,0.787377,-0.036708,0.170271,0.013793,...,-0.597809,-0.063039,-0.046856,-0.457665,-0.002549,-0.240480,-0.478662,0.745601,0.103959,-0.207648
2,0.045846,0.100960,0.821701,-0.607623,-0.284311,-0.441894,0.530290,0.044924,0.023668,-0.153070,...,-0.344162,0.005235,-0.000963,-0.562913,0.004681,-0.224838,0.408748,-0.020797,0.293530,-0.089453
3,-0.027385,0.028644,0.783499,0.066863,-0.630382,-0.210913,-0.370628,0.232392,-0.129108,-0.095263,...,-1.029153,-0.051182,0.353230,0.589289,0.005348,-0.358549,0.460924,-0.037915,0.171848,-0.038181
4,0.011740,-0.048847,0.261959,-0.117680,0.034672,-0.180913,0.306840,-0.308428,-0.282731,-0.209485,...,-0.491192,-0.014284,0.057068,-0.052617,0.007425,-0.161727,-0.224275,-0.035787,0.113925,-0.197622


In [5]:
print(f"NSU nELISA mean of Activin A: {nELISA_data_values['Activin A [NSU]'].mean()}")
print(f"NSU nELISA STDEV of Activin A: {nELISA_data_values['Activin A [NSU]'].std()}")

print(
    f"NSU sensor normalized nELISA mean of Activin A: {nELISA_data_values_sensor_max_norm['Activin A [NSU]'].mean()}"
)
print(
    f"NSU sensor normalized nELISA STDEV of Activin A: {nELISA_data_values_sensor_max_norm['Activin A [NSU]'].std()}"
)

# normalize the data min max
scaler = preprocessing.MinMaxScaler().fit(nELISA_data_values)
nELISA_data_min_max = scaler.transform(nELISA_data_values)

nELISA_data_min_max = pd.DataFrame(
    nELISA_data_min_max, columns=nELISA_data_values.columns
)
print(
    f"NSU Min Max normalized nELISA mean of Activin A: {nELISA_data_min_max['Activin A [NSU]'].mean()}"
)
print(
    f"NSU Min Max normalized nELISA STDEV of Activin A: {nELISA_data_min_max['Activin A [NSU]'].std()}"
)

# normalize the data standard scaler
scaler = preprocessing.StandardScaler().fit(nELISA_data_values)
nELISA_data_standard = scaler.transform(nELISA_data_values)
nELISA_data_standard = pd.DataFrame(
    nELISA_data_standard, columns=nELISA_data_values.columns
)

print(
    f"NSU Standard Scalar normalized nELISA mean of Activin A: {nELISA_data_standard['Activin A [NSU]'].mean()}"
)
print(
    f"NSU Standard Scalar normalized nELISA STDEV of Activin A: {nELISA_data_standard['Activin A [NSU]'].std()}"
)

NSU nELISA mean of Activin A: 0.5217247377399898
NSU nELISA STDEV of Activin A: 5.293904102061589
NSU sensor normalized nELISA mean of Activin A: 0.01537686961452774
NSU sensor normalized nELISA STDEV of Activin A: 0.15602801101944963
NSU Min Max normalized nELISA mean of Activin A: 0.12751334233458655
NSU Min Max normalized nELISA STDEV of Activin A: 0.13825833827735404
NSU Standard Scalar normalized nELISA mean of Activin A: 2.0625333403776493e-17
NSU Standard Scalar normalized nELISA STDEV of Activin A: 1.000726480299234


In [6]:
nELISA_data_values.head()

,Activin A [NSU],AITRL (GITR Ligand) [NSU],Amphiregulin [NSU],Amyloid beta [NSU],APRIL [NSU],BAFF [NSU],BCMA (TNFRSF17) [NSU],BDNF [NSU],BMP2 [NSU],BMP3 [NSU],...,TWEAK [NSU],uPA [NSU],VCAM-1 [NSU],VEGF Receptor 2 (Flk-1) [NSU],VEGF-A (165) [NSU],VEGF-C [NSU],VEGF-D [NSU],VEGFR-1 [NSU],WISP-1 (CCN4) [NSU],XCL1 (Lymphotactin) [NSU]
0,26.491764,-1.073383,25.817862,-2.435479,-2.556473,-1.259130,-0.043782,-0.661045,-1.508780,-2.046371,...,-0.728423,-0.681505,0.124148,2.617804,-2.210675,-1.433347,1.077095,5.713861,0.055628,-0.889479
1,24.943616,0.718127,26.598410,0.926190,-3.238686,-0.387113,3.650604,-0.227975,0.450686,0.104226,...,-1.296642,-1.387073,-0.140898,-1.198079,-0.128332,-1.550067,-2.169066,6.024132,0.340468,-0.844918
2,1.555501,0.268977,27.596171,-2.300495,-3.045141,-1.129626,2.458642,0.278998,0.062647,-1.156700,...,-0.746483,0.115185,-0.002896,-1.473596,0.235711,-1.449242,1.852248,-0.168032,0.961319,-0.363984
3,-0.929140,0.076314,26.313207,0.253148,-6.751763,-0.539164,-1.718383,1.443255,-0.341732,-0.719875,...,-2.232221,-1.126181,1.062176,1.542644,0.269262,-2.311105,2.088687,-0.306334,0.562805,-0.155359
4,0.398325,-0.130139,8.797693,-0.445542,0.371358,-0.462473,1.422635,-1.915471,-0.748352,-1.583011,...,-1.065390,-0.314291,0.171604,-0.137741,0.373863,-1.042447,-1.016304,-0.289143,0.373108,-0.804121


In [7]:
nELISA_data_values_sensor_max_norm.head()

,Activin A [NSU],AITRL (GITR Ligand) [NSU],Amphiregulin [NSU],Amyloid beta [NSU],APRIL [NSU],BAFF [NSU],BCMA (TNFRSF17) [NSU],BDNF [NSU],BMP2 [NSU],BMP3 [NSU],...,TWEAK [NSU],uPA [NSU],VCAM-1 [NSU],VEGF Receptor 2 (Flk-1) [NSU],VEGF-A (165) [NSU],VEGF-C [NSU],VEGF-D [NSU],VEGFR-1 [NSU],WISP-1 (CCN4) [NSU],XCL1 (Lymphotactin) [NSU]
0,0.780796,-0.402891,0.768750,-0.643275,-0.238687,-0.492554,-0.009443,-0.106441,-0.570024,-0.270803,...,-0.335835,-0.030973,0.041286,1.000000,-0.043905,-0.222372,0.237690,0.707199,0.016986,-0.218599
1,0.735167,0.269547,0.791992,0.244632,-0.302382,-0.151433,0.787377,-0.036708,0.170271,0.013793,...,-0.597809,-0.063039,-0.046856,-0.457665,-0.002549,-0.240480,-0.478662,0.745601,0.103959,-0.207648
2,0.045846,0.100960,0.821701,-0.607623,-0.284311,-0.441894,0.530290,0.044924,0.023668,-0.153070,...,-0.344162,0.005235,-0.000963,-0.562913,0.004681,-0.224838,0.408748,-0.020797,0.293530,-0.089453
3,-0.027385,0.028644,0.783499,0.066863,-0.630382,-0.210913,-0.370628,0.232392,-0.129108,-0.095263,...,-1.029153,-0.051182,0.353230,0.589289,0.005348,-0.358549,0.460924,-0.037915,0.171848,-0.038181
4,0.011740,-0.048847,0.261959,-0.117680,0.034672,-0.180913,0.306840,-0.308428,-0.282731,-0.209485,...,-0.491192,-0.014284,0.057068,-0.052617,0.007425,-0.161727,-0.224275,-0.035787,0.113925,-0.197622


In [8]:
nELISA_data_min_max.head()

,Activin A [NSU],AITRL (GITR Ligand) [NSU],Amphiregulin [NSU],Amyloid beta [NSU],APRIL [NSU],BAFF [NSU],BCMA (TNFRSF17) [NSU],BDNF [NSU],BMP2 [NSU],BMP3 [NSU],...,TWEAK [NSU],uPA [NSU],VCAM-1 [NSU],VEGF Receptor 2 (Flk-1) [NSU],VEGF-A (165) [NSU],VEGF-C [NSU],VEGF-D [NSU],VEGFR-1 [NSU],WISP-1 (CCN4) [NSU],XCL1 (Lymphotactin) [NSU]
0,0.805760,0.252327,0.791864,0.201684,0.279975,0.170082,0.458191,0.366062,0.205627,0.267478,...,0.450302,0.080920,0.514189,1.000000,0.000000,0.261275,0.622935,0.784695,0.557588,0.238457
1,0.765328,0.610704,0.812782,0.633036,0.242950,0.359759,0.885877,0.406015,0.580188,0.431526,...,0.342499,0.052334,0.469524,0.390866,0.039617,0.250331,0.268604,0.812933,0.596731,0.245301
2,0.154512,0.520855,0.839522,0.219004,0.253454,0.198251,0.747887,0.452787,0.506013,0.335342,...,0.446875,0.113198,0.492780,0.346884,0.046543,0.259784,0.707547,0.249380,0.682049,0.319165
3,0.089622,0.482315,0.805139,0.546675,0.052290,0.326686,0.264329,0.560197,0.428714,0.368663,...,0.165000,0.062904,0.672261,0.828371,0.047181,0.178978,0.733355,0.236793,0.627285,0.351206
4,0.124291,0.441015,0.335727,0.457023,0.438873,0.343367,0.627953,0.250333,0.350986,0.302823,...,0.386372,0.095798,0.522186,0.560129,0.049171,0.297925,0.394432,0.238358,0.601216,0.251567


In [9]:
nELISA_data_standard.head()

,Activin A [NSU],AITRL (GITR Ligand) [NSU],Amphiregulin [NSU],Amyloid beta [NSU],APRIL [NSU],BAFF [NSU],BCMA (TNFRSF17) [NSU],BDNF [NSU],BMP2 [NSU],BMP3 [NSU],...,TWEAK [NSU],uPA [NSU],VCAM-1 [NSU],VEGF Receptor 2 (Flk-1) [NSU],VEGF-A (165) [NSU],VEGF-C [NSU],VEGF-D [NSU],VEGFR-1 [NSU],WISP-1 (CCN4) [NSU],XCL1 (Lymphotactin) [NSU]
0,4.909214,-1.338097,3.151477,-1.585962,-1.076774,-1.793466,-0.303762,-0.335031,-1.738678,-1.360830,...,-0.169257,-0.822029,0.094377,3.463227,-1.838779,-0.690573,0.929507,4.044421,-0.091531,-0.495617
1,4.616561,0.969283,3.258619,1.522504,-1.337382,-0.566890,2.756890,-0.007488,0.931190,0.016577,...,-0.868352,-0.983196,-0.200601,-0.521682,-1.699131,-0.799740,-1.517003,4.263257,0.170798,-0.442033
2,0.195419,0.390799,3.395577,-1.461146,-1.263447,-1.611306,1.769397,0.375950,0.402468,-0.791016,...,-0.191477,-0.640046,-0.047015,-0.809404,-1.674717,-0.705439,1.513710,-0.104126,0.742583,0.136279
3,-0.274262,0.142658,3.219471,0.900156,-2.679391,-0.780763,-1.691102,1.256510,-0.148518,-0.511240,...,-2.019419,-0.923603,1.138334,2.340442,-1.672467,-1.511534,1.691905,-0.201671,0.375564,0.387145
4,-0.023327,-0.123244,0.815197,0.254092,0.041670,-0.672891,0.911106,-1.283789,-0.702557,-1.064058,...,-0.583837,-0.738148,0.147191,0.585624,-1.665452,-0.324967,-0.648210,-0.189546,0.200858,-0.392976


In [10]:
def umap_graph(df, df1, save_name):
    # UMAP parameters
    umap_df = umap.UMAP(
        n_neighbors=6,
        min_dist=0.8,
        n_components=2,
        metric="cosine",
        spread=1.1,
        init="random",
        random_state=0,
    )

    # fit and transform data
    proj_2d = umap_df.fit_transform(df)

    # plot UMAP
    fig_2d = px.scatter(
        proj_2d,
        x=0,
        y=1,
        color=df1.cell_type,
        labels={"color": "Cell Type"},
        title="UMAP projection of the nELISA data",
    ).update_layout(xaxis_title="UMAP_1", yaxis_title="UMAP_2")

    fig_2d.update_traces(marker={"size": 15})
    fig_2d.show()

    # save UMAP
    fig_2d.write_html(f"{save_name}.html")
    fig_2d.write_image(f"{save_name}.png")


umap_graph(nELISA_data_values, nELISA_data, "./figures/nELISA_data_values_raw")
umap_graph(
    nELISA_data_values_sensor_max_norm,
    nELISA_data,
    "./figures/nELISA_data_values_max_sensor_noramalized",
)
umap_graph(nELISA_data_min_max, nELISA_data, "./figures/nELISA_data_min_max_normalized")
umap_graph(
    nELISA_data_standard, nELISA_data, "./figures/nELISA_data_standard_normalized"
)